# Traffic Light Detection with YOLO on Bosch data set

This script tries to detect traffic lights in the _Bosch Small Traffic Lights Dataset_ with the [YOLO v2 CNN](https://pjreddie.com/darknet/yolo). To make use of tensorflow on Udacity's carla, the github tensorflow implementation made by thtrieu will be used.

To run this script make sure to meet the following preconditions:
1. Download the train data from https://hci.iwr.uni-heidelberg.de/node/6132/download/c2267f98dcef13de059fa8cdb0f8efed.
    - dataset_train_rgb.zip.001	to dataset_train_rgb.zip.004
    - unzip them all together, you will get a _rgb_ directory with the training data
    - the _rgb_ folder and _train.yaml_ has to be in the _TrafficLightDetector/data_ directory
2. Clone the _darkflow_ repo on the same level as _TrafficLightDetection_
    - https://github.com/thtrieu/darkflow.git
    - install as described (you might need to restart the notebook to have access)
    ```
    pip install -e .
    ```
3. In the _darkflow_ repo, run __/test/test_darkflow.py__ to download the pretraind weights-files
    - check if _/bin/tiny-yolo-voc.weights_ exists
    
Your directories should look like this (or if not, you might have to fix some relative paths in the code below):

- TrafficLightDetector
    - cfg
        - tiny-yolo-voc.cfg
        - yolo_bosch.cfg
    - ckpt
    - data
        - rgb
        - train.yaml
    
    - annotations (<- created below)   
    - ...
- darkflow
    - bin
    - darkflow
    - test
    - ...
    
---    
#### Dependencies:
- tensorflow 1.0
- cv2
- lxml
- tqdm
- yaml

In [1]:
import os
import yaml
from lxml import etree as ET
from tqdm import tqdm
from random import shuffle

First, we need to load the data sets annotations from the _train.yaml_ file in the data folder.

In [2]:
data = []
with open("./data/train.yaml", 'r') as stream:
    try:
        data = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)
        
shuffle(data)

Darkflow uses the Pascal VOC annotation format. The next step converts our annotations to this very format.
### Caution: One annotation has an error (ymin > ymax). The training will fail if you don't eliminate it.
#### Check the output below and switch the min and max  values in _train.yaml_ for the faulty annotation. Then run this cell again.

In [3]:
annotation_path = './annotations'

if not os.path.isdir(annotation_path):
    os.makedirs(annotation_path)

TRAINING_DATA = 1500
#TRAINING_DATA = len(data)
cnt = 0

label_count = dict(zip([
    'Red',
    'RedStraight',
    'RedStraightLeft',
    'RedLeft',
    'RedRight',
    'Yellow',
    'Green',
    'GreenStraight',
    'GreenStraightLeft',
    'GreenStraightRight',
    'GreenLeft',
    'GreenRight',
    'off'],
    [0]*13
))

for img in tqdm(data[:TRAINING_DATA], ncols=80):
    root = ET.Element('annotation')
    
    fn = ET.SubElement(root, 'filename')
    fn.text = img['path']
    
    sz = ET.SubElement(root, 'size')
    
    width = ET.SubElement(sz, 'width')
    width.text = '1280'
    
    height = ET.SubElement(sz, 'height')
    height.text = '720'
    
    depth = ET.SubElement(sz, 'depth')
    depth.text = '3'

    for box in img['boxes']:       
        obj = ET.SubElement(root, 'object')
        
        name = ET.SubElement(obj, 'name')
        name.text = box['label']
        
        bb = ET.SubElement(obj, 'bndbox')
        
        xmin = ET.SubElement(bb, 'xmin')
        x_min = box['x_min']
        xmin.text = str(box['x_min'])
        
        xmax = ET.SubElement(bb, 'xmax')
        x_max = box['x_max']
        xmax.text = str(box['x_max'])
        if (x_max - x_min) < 0:
            print("Annotation Error X", img['path'])
        
        ymin = ET.SubElement(bb, 'ymin')
        y_min = box['y_min']
        ymin.text = str(box['y_min'])
        
        ymax = ET.SubElement(bb, 'ymax')
        y_max = box['y_max']
        ymax.text = str(box['y_max'])
        
        if (y_max - y_min) < 0:
            print("Annotation Error Y", img['path'])
        
        label_count[box['label']] += 1

    et = ET.ElementTree(root)
    et.write(os.path.join(annotation_path, str(cnt) + '.xml'), pretty_print=True)
    
    cnt += 1

for label, count in label_count.items():
    print('{:>20}'.format(label), count)

100%|█████████████████████████████████████| 1500/1500 [00:00<00:00, 2315.97it/s]


             RedLeft 335
          GreenRight 1
   GreenStraightLeft 0
               Green 1661
            RedRight 1
                 Red 846
     RedStraightLeft 0
       GreenStraight 8
                 off 229
         RedStraight 2
           GreenLeft 45
  GreenStraightRight 1
              Yellow 135


### Training

Before training make sure
- that the _labels.txt_ file is in the same directory as this notebook
- annotations exist in Pascal VOC format (many xml files named like this -> 0.xml, 1.xml, 2.xml, ...)
- you have executed darkflow/test/test_darkflow.py successfully and the file darkflow/bin/tiny-yolo-voc.weights
- you have a gpu, if not delete the _gpu_ statement in _options_ to run on cpu

In [4]:
import darkflow
from darkflow.net.build import TFNet
import cv2

options = {"model": "./cfg/yolo_bosch.cfg",
           "load": "../darkflow/bin/tiny-yolo-voc.weights",
           "train": True, 
           "annotation": "./annotations/",
           "dataset": "./data/",
           "batch": 4, "trainer": "adam",
           "epoch": 4,
           "lrate": 1e-5,
           "gpu": 1.0}

tfnet = TFNet(options)

tfnet.train()

Parsing ./cfg/tiny-yolo-voc.cfg
Parsing ./cfg/yolo_bosch.cfg
Loading ../darkflow/bin/tiny-yolo-voc.weights ...
Successfully identified 63471556 bytes
Finished in 0.02215266227722168s

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 416, 416, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 416, 416, 16)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 208, 208, 16)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 208, 208, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 104, 104, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 52, 52, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 52, 52, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 26, 26, 128)
 Load  |  Yep!  | conv 3x3p1_1  

step 83 - loss 71.95941925048828 - moving ave loss 75.7794899278824
step 84 - loss 70.21585845947266 - moving ave loss 75.22312678104143
step 85 - loss 70.84208679199219 - moving ave loss 74.78502278213651
step 86 - loss 71.35968017578125 - moving ave loss 74.44248852150098
step 87 - loss 70.8161392211914 - moving ave loss 74.07985359147003
step 88 - loss 70.53596496582031 - moving ave loss 73.72546472890505
step 89 - loss 69.48411560058594 - moving ave loss 73.30132981607314
step 90 - loss 69.77369689941406 - moving ave loss 72.94856652440724
step 91 - loss 67.6751480102539 - moving ave loss 72.4212246729919
step 92 - loss 69.99752044677734 - moving ave loss 72.17885425037045
step 93 - loss 69.01957702636719 - moving ave loss 71.86292652797013
step 94 - loss 67.96820068359375 - moving ave loss 71.4734539435325
step 95 - loss 67.01248168945312 - moving ave loss 71.02735671812457
step 96 - loss 66.73036193847656 - moving ave loss 70.59765724015976
step 97 - loss 65.6294937133789 - movin

step 200 - loss 40.16737365722656 - moving ave loss 42.00469380596118
step 201 - loss 39.521915435791016 - moving ave loss 41.75641596894417
step 202 - loss 36.6663932800293 - moving ave loss 41.24741370005268
step 203 - loss 42.92328643798828 - moving ave loss 41.41500097384624
step 204 - loss 37.674503326416016 - moving ave loss 41.04095120910322
step 205 - loss 36.19020080566406 - moving ave loss 40.555876168759305
step 206 - loss 40.332923889160156 - moving ave loss 40.53358094079939
step 207 - loss 39.1485710144043 - moving ave loss 40.39507994815988
step 208 - loss 41.42572784423828 - moving ave loss 40.498144737767724
step 209 - loss 39.03424835205078 - moving ave loss 40.35175509919603
step 210 - loss 37.90156173706055 - moving ave loss 40.10673576298248
step 211 - loss 36.86126708984375 - moving ave loss 39.78218889566861
step 212 - loss 39.09956741333008 - moving ave loss 39.713926747434755
step 213 - loss 37.761940002441406 - moving ave loss 39.51872807293542
step 214 - loss

step 315 - loss 27.15247917175293 - moving ave loss 26.676157961139264
step 316 - loss 24.792245864868164 - moving ave loss 26.487766751512154
step 317 - loss 24.419143676757812 - moving ave loss 26.28090444403672
step 318 - loss 24.360471725463867 - moving ave loss 26.08886117217944
step 319 - loss 24.28094482421875 - moving ave loss 25.908069537383373
step 320 - loss 25.245820999145508 - moving ave loss 25.841844683559586
step 321 - loss 28.66678237915039 - moving ave loss 26.124338453118668
step 322 - loss 25.857091903686523 - moving ave loss 26.097613798175455
step 323 - loss 27.461700439453125 - moving ave loss 26.234022462303223
step 324 - loss 24.923336029052734 - moving ave loss 26.102953818978175
step 325 - loss 23.136972427368164 - moving ave loss 25.806355679817173
step 326 - loss 23.392635345458984 - moving ave loss 25.564983646381357
step 327 - loss 23.775146484375 - moving ave loss 25.385999930180724
step 328 - loss 26.716360092163086 - moving ave loss 25.519035946378963


step 429 - loss 21.22083282470703 - moving ave loss 19.12832794122252
step 430 - loss 19.58156967163086 - moving ave loss 19.17365211426335
step 431 - loss 18.84554672241211 - moving ave loss 19.140841575078227
step 432 - loss 18.283008575439453 - moving ave loss 19.055058275114348
step 433 - loss 16.840524673461914 - moving ave loss 18.833604914949106
step 434 - loss 18.656322479248047 - moving ave loss 18.815876671379
step 435 - loss 21.880184173583984 - moving ave loss 19.1223074215995
step 436 - loss 17.750383377075195 - moving ave loss 18.985115017147074
step 437 - loss 19.48525619506836 - moving ave loss 19.0351291349392
step 438 - loss 18.288345336914062 - moving ave loss 18.960450755136687
step 439 - loss 15.606462478637695 - moving ave loss 18.625051927486787
step 440 - loss 19.893413543701172 - moving ave loss 18.751888089108228
step 441 - loss 19.50928497314453 - moving ave loss 18.82762777751186
step 442 - loss 16.406429290771484 - moving ave loss 18.585507928837824
step 44

...

step 1002 - loss 9.837743759155273 - moving ave loss 9.505495173735495
step 1003 - loss 10.692587852478027 - moving ave loss 9.624204441609749
step 1004 - loss 8.684696197509766 - moving ave loss 9.530253617199751
step 1005 - loss 8.182175636291504 - moving ave loss 9.395445819108927
step 1006 - loss 14.081461906433105 - moving ave loss 9.864047427841346
step 1007 - loss 7.582281112670898 - moving ave loss 9.635870796324301
step 1008 - loss 9.086324691772461 - moving ave loss 9.580916185869118
step 1009 - loss 7.112797260284424 - moving ave loss 9.334104293310649
step 1010 - loss 5.672411918640137 - moving ave loss 8.967935055843597
step 1011 - loss 7.140444278717041 - moving ave loss 8.785185978130942
step 1012 - loss 14.476428985595703 - moving ave loss 9.354310278877419
step 1013 - loss 6.999467849731445 - moving ave loss 9.118826035962822
step 1014 - loss 8.002466201782227 - moving ave loss 9.007190052544763
step 1015 - loss 7.335894584655762 - moving ave loss 8.840060505755863
ste

step 1116 - loss 5.1029276847839355 - moving ave loss 8.943643121411931
step 1117 - loss 5.6400885581970215 - moving ave loss 8.613287665090441
step 1118 - loss 7.9484968185424805 - moving ave loss 8.546808580435645
step 1119 - loss 12.390486717224121 - moving ave loss 8.931176394114493
step 1120 - loss 7.519107818603516 - moving ave loss 8.789969536563396
step 1121 - loss 4.528887748718262 - moving ave loss 8.363861357778882
step 1122 - loss 6.650932312011719 - moving ave loss 8.192568453202167
step 1123 - loss 11.443817138671875 - moving ave loss 8.517693321749137
step 1124 - loss 10.028063774108887 - moving ave loss 8.668730366985113
step 1125 - loss 7.444459915161133 - moving ave loss 8.546303321802714
Finish 3 epoch(es)
step 1126 - loss 8.652013778686523 - moving ave loss 8.556874367491096
step 1127 - loss 9.164424896240234 - moving ave loss 8.61762942036601
step 1128 - loss 9.744048118591309 - moving ave loss 8.73027129018854
step 1129 - loss 15.780874252319336 - moving ave loss 

step 1230 - loss 12.143238067626953 - moving ave loss 10.160799137947189
step 1231 - loss 7.442503452301025 - moving ave loss 9.888969569382573
step 1232 - loss 7.104869842529297 - moving ave loss 9.610559596697245
step 1233 - loss 16.10858154296875 - moving ave loss 10.260361791324396
step 1234 - loss 8.039432525634766 - moving ave loss 10.038268864755432
step 1235 - loss 7.152187347412109 - moving ave loss 9.749660713021099
step 1236 - loss 7.967520236968994 - moving ave loss 9.57144666541589
step 1237 - loss 6.766911506652832 - moving ave loss 9.290993149539583
step 1238 - loss 9.863945960998535 - moving ave loss 9.348288430685477
step 1239 - loss 10.187796592712402 - moving ave loss 9.43223924688817
step 1240 - loss 13.392101287841797 - moving ave loss 9.828225450983533
step 1241 - loss 8.758891105651855 - moving ave loss 9.721292016450366
step 1242 - loss 18.883235931396484 - moving ave loss 10.63748640794498
step 1243 - loss 5.96798038482666 - moving ave loss 10.170535805633149
s

step 1344 - loss 3.8837666511535645 - moving ave loss 9.345523935786114
step 1345 - loss 6.313348770141602 - moving ave loss 9.042306419221662
step 1346 - loss 11.098194122314453 - moving ave loss 9.247895189530942
step 1347 - loss 6.832771301269531 - moving ave loss 9.0063828007048
step 1348 - loss 7.17290735244751 - moving ave loss 8.823035255879072
step 1349 - loss 11.847370147705078 - moving ave loss 9.125468745061672
step 1350 - loss 21.185588836669922 - moving ave loss 10.331480754222497
step 1351 - loss 4.663103103637695 - moving ave loss 9.764642989164019
step 1352 - loss 7.950299263000488 - moving ave loss 9.583208616547667
step 1353 - loss 10.308107376098633 - moving ave loss 9.655698492502765
step 1354 - loss 7.764311790466309 - moving ave loss 9.46655982229912
step 1355 - loss 8.641754150390625 - moving ave loss 9.38407925510827
step 1356 - loss 9.16230297088623 - moving ave loss 9.361901626686066
step 1357 - loss 6.984357833862305 - moving ave loss 9.124147247403691
step 1

step 1458 - loss 21.753793716430664 - moving ave loss 13.754869026860977
step 1459 - loss 8.407529830932617 - moving ave loss 13.220135107268142
step 1460 - loss 9.765371322631836 - moving ave loss 12.874658728804512
step 1461 - loss 11.681072235107422 - moving ave loss 12.755300079434802
step 1462 - loss 17.88754653930664 - moving ave loss 13.268524725421987
step 1463 - loss 12.36636734008789 - moving ave loss 13.178308986888577
step 1464 - loss 8.13913631439209 - moving ave loss 12.67439171963893
step 1465 - loss 14.085830688476562 - moving ave loss 12.815535616522693
step 1466 - loss 13.229072570800781 - moving ave loss 12.856889311950503
step 1467 - loss 6.935234069824219 - moving ave loss 12.264723787737875
step 1468 - loss 8.346435546875 - moving ave loss 11.872894963651587
step 1469 - loss 6.579625129699707 - moving ave loss 11.343567980256399
step 1470 - loss 12.208730697631836 - moving ave loss 11.430084251993943
step 1471 - loss 8.640497207641602 - moving ave loss 11.15112554